<a href="https://colab.research.google.com/github/owen-YN/DWH_FINALPROJECT_3DSA_GROUP1_Rogue_One/blob/main/ETL_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from dateutil import parser
import matplotlib.pyplot as plt
from google.colab import files

In [11]:
from google.colab import files
uploaded = files.upload()

Saving dirty_cafe_sales.csv to dirty_cafe_sales (1).csv


In [23]:
def check_data(df):
    print("Data info:")
    df.info()  # prints directly in Colab

    print("\nMissing values:")
    display(df.isna().sum())

    print("\nDuplicate rows:", df.duplicated().sum())

# run check
check_data(df)


Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              10000 non-null  object
 2   Quantity          10000 non-null  object
 3   Price Per Unit    10000 non-null  object
 4   Total Spent       10000 non-null  object
 5   Payment Method    10000 non-null  object
 6   Location          10000 non-null  object
 7   Transaction Date  10000 non-null  object
dtypes: object(8)
memory usage: 625.1+ KB

Missing values:


,0
Transaction ID,0
Item,0
Quantity,0
Price Per Unit,0
Total Spent,0
Payment Method,0
Location,0
Transaction Date,0



Duplicate rows: 0


In [24]:
def clean_dirty_cafe_data(df):

    # Strip column names
    df.columns = df.columns.str.strip()

    # Trim whitespace in string columns
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str).str.strip()

    # Replace bad values
    df = df.replace({
        "ERROR": np.nan,
        "UNKNOWN": np.nan,
        "": np.nan
    })

    # Convert numeric fields
    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
    df["Price Per Unit"] = pd.to_numeric(df["Price Per Unit"], errors="coerce")
    df["Total Spent"] = pd.to_numeric(df["Total Spent"], errors="coerce")

    # Convert date
    df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

    # Drop rows missing key values
    df = df.dropna(subset=["Item", "Quantity", "Price Per Unit", "Transaction Date"])

    # Fix revenue
    df["Revenue"] = df["Quantity"] * df["Price Per Unit"]

    print("Cleaned dataset:")
    display(df.head())
    print("\nRemaining missing values:")
    display(df.isna().sum())

    return df

df_clean = clean_dirty_cafe_data(df)


Cleaned dataset:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Revenue
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08,4.0
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16,12.0
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19,4.0
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27,10.0
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11,4.0



Remaining missing values:


,0
Transaction ID,0
Item,0
Quantity,0
Price Per Unit,0
Total Spent,401
Payment Method,483
Location,561
Transaction Date,0
Revenue,0


1. What is the total revenue generated by each item?

In [26]:
total_revenue = df_clean.groupby("Item")["Revenue"].sum().sort_values(ascending=False)
display(total_revenue)


,Revenue
Item,
Salad,14890.0
Sandwich,11680.0
Smoothie,11608.0
Juice,9108.0
Cake,9009.0
Coffee,6210.0
Tea,4179.0
Cookie,2775.0
nan,2481.0


2. What is the most ordered item?


In [27]:
most_ordered = df_clean.groupby("Item")["Quantity"].sum().sort_values(ascending=False)
display(most_ordered)

,Quantity
Item,
Coffee,3105.0
Juice,3036.0
Cake,3003.0
Salad,2978.0
Sandwich,2920.0
Smoothie,2902.0
Tea,2786.0
Cookie,2775.0
nan,830.0


3. On which day of the week does the cafe see the highest sales volume?

In [28]:
df_clean["Day"] = df_clean["Transaction Date"].dt.day_name()
day_sales = df_clean.groupby("Day")["Quantity"].sum().sort_values(ascending=False)
display(day_sales)

,Quantity
Day,
Sunday,3565.0
Monday,3545.0
Friday,3533.0
Thursday,3500.0
Saturday,3474.0
Tuesday,3417.0
Wednesday,3301.0


4. What are the revenue trends across different months?

In [35]:
df_clean["MonthName"] = df_clean["Transaction Date"].dt.strftime("%b")
monthly_revenue_sorted = monthly_revenue.sort_values(ascending=False)

display(monthly_revenue_sorted)

,,Revenue
MonthNum,MonthName,
6,Jun,6296.0
10,Oct,6208.5
1,Jan,6093.0
11,Nov,6077.0
4,Apr,6053.0
3,Mar,6042.0
8,Aug,5995.0
12,Dec,5972.0
7,Jul,5955.0
